In [30]:
import pandas as pd
import numpy as np

data_raw = pd.read_csv('../../전처리된 데이터/지역사회건강조사 1차 전처리_2019.csv')

df = data_raw[['만나이','성별','행복감 지수','자치구']]

df_hi1=df[df['행복감 지수'] != 99] # 99는 모름
df_hi2=df_hi1[df_hi1['행복감 지수'] != 77] # 77은 응답거부

In [31]:
data_pbhlth_code = pd.read_excel('../../raw/지역사회건강조사 2019년 원시자료 이용지침서 등/지역사회건강조사 2019 원시자료 참고사항_보건소정보, 세대유형.xlsx',header=3)
data_pbhlth_code = data_pbhlth_code[data_pbhlth_code['시도코드'] == 11][['보건소코드','보건소명']]
data_pbhlth_code["보건소명"] = data_pbhlth_code["보건소명"].str.replace('보건소', '', regex=True)
data_pbhlth_code.columns=['보건소코드','자치구']

list_region = list(data_pbhlth_code.to_dict()['자치구'].values()) + ['서울시'] # 전체 합에 해당하는 서울시 추가
list_region.reverse() # 서울시가 맨 위에 올라오도록 reverse 사용

In [32]:
list_df = [] # append를 이용해 삽입하기 위해서 빈 리스트 선언
for value in list_region:
    temp = df_hi2['자치구'] == value
    
    # list_region에 들어있는 자치구와 df_hi2에 있는 자치구와 동일하면 temp에는 True과 들어가고, 같지 않으면 False
    list_df.append(df_hi2[temp]) # 그래서 temp가 true이면 value와 동일한 자치구를 가진 데이터프레임을 리스트에 넣는다
    
list_df[0] = df_hi2
# df_hi2에서 자치구 column에는 서울시가 없기 때문에 list_df[0]은 빈 데이터프레임이 들어있는데 여기에 전체 데이터를 넣어준다.

In [34]:
list_age = ['19~200','19~44','45~64','65~74','75~200'] # 전체 / 19~44  45~64 / 65~74 / 75이상 으로 나누기 위해서 범위 설정
list_sex = ['계','남','여']
idx_sex = []
idx_age = []
idx_region = []
value_hi = []
# 크게 자치구 -> 연령대 -> 성별로 하위 인덱스를 만드식으로 작업하면서 계(전체)에 해당하는 값을 살리려고 함
for i, value1 in enumerate(list_region): # enumerate를 이용해 list_df의 인덱스와 자치구로 이용
    for j in list_age:
        for k in list_sex:            
            temp = list_df[i][list_df[i]['만나이']>=int(j.split('~')[0])] 
            temp1 = temp[temp['만나이'] <= int(j.split('~')[1])] # 연령대를 ~를 기준으로 이상, 이하로 데이터 처리
            
            # 이 부분은 위에 list_age에서 넘어오는 계 / 남 / 여를 구분해서 만들기 위해 아래와 같이 조건문 사용
            if k == '남':
                temp2 = temp1[temp1['성별'] == 1]
            elif k == '여':
                temp2 = temp1[temp1['성별'] == 2]
            else:
                temp2 = temp1
                
            # 행복감 지수는 범주형 데이터여서 value_counts()롤 각각의 값에 해당하는 데이터들의 갯수를 세어서 딕셔너리로 만듬
            temp3 = temp2['행복감 지수'].value_counts().to_dict()
            # 행복감 지수는 1부터 10까지 이루어져있지만, 설문조사상 해당하는 값이 없는 경우는 딕셔너리가 만들어지지 않음
            # 그래서 없는 값도 생성 후 0으로 넣어서 추후 전처리 작업 진행간에 오류 발생 방지
            for m in range(1,11): 
                if m not in temp3: 
                    temp3[m] = 0
                    
            temp_sum = 0
            # 이 구간에서 없는 값은 오류가 발생할 수도 있어서 위에서 방지
            # 그리고 각 구간별 수와 값을 곱해서 계속해서 더한 후 나눠서, 젠처 평균을 구함
            for n in range(1,11):
                temp_sum += temp3[n]*n  
            value_hi.append(round(temp_sum / len(temp2['행복감 지수']),2))
            
            # 아래 작업은 멀티 인덱스를 생성하기 위해서 순서대로 idx 리스트에 해당하는 값을 삽입
            if j == '19~200':
                idx_age.append('계')
            elif j == '75~200':
                idx_age.append('75 이상')
            else:
                idx_age.append(j)
            idx_sex.append(k)
            idx_region.append(value1)

In [38]:
df_final = pd.DataFrame({'행복감 지수':value_hi, '지역':idx_region,'연령별':idx_age,'성별':idx_sex})

df_final = df_final.set_index(['지역','연령별','성별'])

df_final.to_csv("../../전처리된 데이터/지역사회건강조사 (행복감 지수)_2019.csv", mode='w',encoding = 'utf-8-sig')

In [39]:
df_final

행복감 지수
지역  연령별   성별        
서울시 계     계     7.01
          남     7.05
          여     6.98
    19~44 계     7.12
          남     7.17
...              ...
강남구 65~74 남     7.10
          여     7.45
    75 이상 계     6.98
          남     7.00
          여     6.97

[390 rows x 1 columns]